<a href="https://colab.research.google.com/github/Sa811-ops/5731-Computational-/blob/main/In_class_exercises_4_Text_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [1]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [6]:
# Q1 — ANSWER CELL
import pandas as pd

FILE_PATH = "/content/support_messages (2).txt"

# Load the TAB-separated file into df
df = pd.read_csv(FILE_PATH, sep="\t", names=["id", "message"])

# Print shape and first 3 rows
print(df.shape)
print(df.head(3))

(9, 2)
   id                                            message
0  id                                            message
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...
2   2  Refund please!!! I was charged 2 times... invo...


## Q2 (3 points) — Answer below

In [7]:
# Q2 — ANSWER CELL
# create word_count, char_count, num_count, upper_word_count

import re

# word_count → number of words
df["word_count"] = df["message"].str.split().str.len()

# char_count → number of characters
df["char_count"] = df["message"].str.len()

# num_count → number of digits (0–9)
df["num_count"] = df["message"].str.count(r"\d")

# upper_word_count → ALL-CAPS words like WHY, DAMAGED
df["upper_word_count"] = df["message"].apply(
    lambda x: sum(token.isupper() for token in re.findall(r"\b\w+\b", str(x)))
)

# display full DataFrame
print(df)

   id                                            message  word_count  \
0  id                                            message           1   
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...          12   
2   2  Refund please!!! I was charged 2 times... invo...          11   
3   3        Great service, thanks! arrived in 2 days :)           8   
4   4  WHY is my package DAMAGED??? tracking says del...           8   
5   5  Need to change address: 7421 Frankford Rd Apt ...          12   
6   6  Support ticket: ORD-1050. Call me at (469) 555...           9   
7   7  I can’t login— password reset link not working...          10   
8   8  Item missing from box. pls send replacement!! ...          10   

   char_count  num_count  upper_word_count  
0           7          0                 0  
1          73          4                 2  
2          71          9                 3  
3          43          1                 0  
4          55          0                 2  
5        

## Q3 (3 points) — Answer below

In [14]:
# Fix id column type (handles strings like " 1", and the header row becomes NaN)
df["id"] = pd.to_numeric(df["id"], errors="coerce")

for target_id in [1, 4]:
    row = df.loc[df["id"] == target_id]

    if row.empty:
        print(f"id={target_id} not found. Available ids:", df["id"].dropna().astype(int).tolist())
    else:
        print(f"\nid={target_id}")
        print("original:", row.iloc[0]["message"])
        print("clean   :", row.iloc[0]["clean"])


id=1
original: Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com
clean   : hi order late order ord 1042 email sara ali gmail com

id=4
original: WHY is my package DAMAGED??? tracking says delivered...
clean   : package damaged tracking says delivered


## Q4 (2 points) — Answer below

In [12]:
# Q4 — ANSWER CELL

import re

# patterns
order_pat = re.compile(r"ORD-\d{4}", re.IGNORECASE)
email_pat = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

# extract first match or None
df["order_id"] = df["message"].apply(
    lambda x: order_pat.search(str(x)).group(0).upper() if order_pat.search(str(x)) else None
)

df["email"] = df["message"].apply(
    lambda x: email_pat.search(str(x)).group(0) if email_pat.search(str(x)) else None
)

# display result
print(df[["id", "order_id", "email"]])

   id  order_id                  email
0  id      None                   None
1   1  ORD-1042     sara.Ali@gmail.com
2   2  ORD-1042                   None
3   3      None                   None
4   4      None                   None
5   5      None                   None
6   6  ORD-1050                   None
7   7      None  mehri.sattari@unt.edu
8   8  ORD-1060                   None


## Q5 (1 point) — Answer below

In [10]:
# Q5 — ANSWER CELL

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 1) fit TF-IDF on clean text
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean"].fillna(""))

# 2) compute average TF-IDF per term across documents
avg_tfidf = np.asarray(X.mean(axis=0)).ravel()
terms = np.array(vectorizer.get_feature_names_out())

# 3) get top 5 terms
top5_idx = np.argsort(avg_tfidf)[::-1][:5]

print("Top 5 keywords by average TF-IDF:\n")
for idx in top5_idx:
    print(f"{terms[idx]}: {avg_tfidf[idx]:.6f}")

Top 5 keywords by average TF-IDF:

ord: 0.125373
message: 0.111111
order: 0.088427
1042: 0.063192
email: 0.058799


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
